In [ ]:
import numpy as np
import time

In [ ]:
def lu(matrix: np.array):
    # taking the length of the row (square matrix)
    n = len(matrix)
    # taking the length of the column
    n1 = len(matrix[0])
    if n != n1:
        return -1  # Return -1 if the matrix is not square

    # initializing P, L, U matrices
    P = np.eye(n)  # P is an identity matrix to start with
    L = np.eye(n)  # L is initialized as an identity matrix
    U = matrix.copy()  # U is a copy of the original matrix

    # Start measuring the execution time
    start_time_LU = time.time()
    for i in range(n-1):
        # finding the largest element column-wise
        pivot_index = np.argmax(np.abs(U[i:, i])) + i  # i is added to correct the index of the matrix

        # partial pivoting (switch the initial row with the pivot row to arrange the increasing order of the pivot element)
        U[[i, pivot_index], :] = U[[pivot_index, i], :]
        L[[i, pivot_index], :] = L[[pivot_index, i], :]
        P[[i, pivot_index], :] = P[[pivot_index, i], :]

        # Gauss elimination
        for j in range(i+1, n):
            factor = U[j, i] / U[i, i]
            L[j, i] = factor  # Store the multiplier in the L matrix
            U[j, i:] = U[j, i:] - (factor * U[i, i:])  # Apply the row operation
    end_time_LU = time.time()

    return P, L, U,start_time_LU,end_time_LU



In [ ]:
def Cholesky(A):
    # Get the size of the matrix A
    n = len(A)

    # Initialize the lower triangular matrix L
    L = np.zeros((n, n), dtype=float)

    start_time_cho=time.time()
    # Perform Cholesky decomposition
    for i in range(n):
        for k in range(i):  # Loop to calculate diagonal elements of L
            L[i, i] += L[i, k]**2
        L[i, i] = np.sqrt(A[i, i] - L[i, i])

        for j in range(i+1, n):  # Loop to calculate non-diagonal elements of L
            for k in range(i):
                L[j, i] += L[j, k] * L[i, k]
            L[j, i] = (A[j, i] - L[j, i]) / L[i, i]
    end_time_cho = time.time()
    return L,start_time_cho,end_time_cho


In [ ]:
def projection(x: np.array, y: np.array):
  return (np.dot(x, y) / np.linalg.norm(y)**2) * y

def gram_schmidt(x: np.array):
    # Check if there's only one vector; no orthogonalization needed
    if len(x) == 1:
        return x

    n = len(x)  # Number of rows
    m = len(x[0])  # Number of columns

    # Initialize matrices for orthogonalized vectors (e) and intermediate vectors (u)
    e = np.zeros((n, m))
    u = np.zeros((n, m))

    # Set the first vector in u and normalize to get the first vector in e
    u[:, 0] = x[:, 0]
    e[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

    # Iterate through the remaining vectors
    for i in range(1, m):
        # Copy the original vector to u
        u[:, i] = x[:, i]

        # Subtract the projections onto previously orthogonalized vectors
        for j in range(i):
            u[:, i] -= projection(x[:, i], u[:, j])

        # Handle the case where the vector becomes a zero vector
        if np.linalg.norm(u[:, i]) > 0:
            # Normalize and store the orthogonalized vector in e
            e[:, i] = u[:, i] / np.linalg.norm(u[:, i])
        else:
            # If the vector is zero, set the corresponding vector in e to zero
            e[:, i] = 0

    return e

def QR(A: np.array):
    n = len(A)
    m = len(A[0])

    # Initialize the upper triangular matrix R
    R = np.zeros((n, m), dtype=float)

    start_time_QR=time.time()
    # Perform Gram-Schmidt orthogonalization
    e = gram_schmidt(A)


    # Compute the entries of R
    for i in range(n):
        for j in range(i, m):
            R[i, j] = np.dot(A[:, j], e[:, i])

    end_time_QR=time.time()
    return e, R,start_time_QR,end_time_QR


In [ ]:
#matrix generation
# Create a symmetric positive definite matrix
A = np.random.rand(50, 50)
symmetric_matrix = 0.5 * (A + A.T)
positive_definite_matrix = symmetric_matrix @ symmetric_matrix.T

# Ensure it's symmetric
positive_definite_matrix = 0.5 * (positive_definite_matrix + positive_definite_matrix.T)

print(positive_definite_matrix)


[[14.16696379 12.54881591 13.59345859 ... 11.70977311 11.96296881
  12.18012083]
 [12.54881591 14.27691873 13.30663563 ... 11.33309055 12.92158011
  12.14991131]
 [13.59345859 13.30663563 17.1484799  ... 12.90730676 13.74061702
  13.90125188]
 ...
 [11.70977311 11.33309055 12.90730676 ... 12.95323381 11.61153485
  11.52410412]
 [11.96296881 12.92158011 13.74061702 ... 11.61153485 15.29716358
  12.4931645 ]
 [12.18012083 12.14991131 13.90125188 ... 11.52410412 12.4931645
  14.58241514]]


In [ ]:
# Calculate ratios
ratio_cho_to_LU = (end_time_cho - start_time_cho) / (end_time_LU - start_time_LU)

# Print the ratio
print("cholesky to LU Ratio:", ratio_cho_to_LU)


cholesky to LU Ratio: 2.202528000921367


In [ ]:
# Calculate ratios
ratio_cho_to_QR = (end_time_cho - start_time_cho) / (end_time_QR - start_time_QR)

# Print the ratio
print("cholesky to QR Ratio:", ratio_cho_to_QR)

cholesky to QR Ratio: 0.6805996708038614
